In [1]:
import os
import glob
from tqdm import tqdm
from langchain_community.document_loaders import UnstructuredHTMLLoader
import pickle
import torch
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
import time
from langchain_community.vectorstores import FAISS
import os
import shutil
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage import LocalFileStore

/home/agrawal.bi/.local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-28 21:28:23.795167: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 21:28:23.795212: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 21:28:23.795254: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 21:28:23.808148: I tensorflow/core/platform/cpu_feat

## Reading saved documents

In [2]:
# Define the path
path = '../documents/processed_docs/pandas_docs.pkl'

# Load data from a file
with open(path, 'rb') as file:
    pandas_docs = pickle.load(file)

In [3]:
#print(pandas_docs[134].page_content)

In [4]:
pandas_docs[134]

Document(metadata={'source': 'https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html', 'title': 'pandas.DataFrame.idxmax — pandas 2.2.2 documentation'}, page_content='pandas.DataFrame.idxmax#\n\nDataFrame.idxmax(axis=0,skipna=True,numeric_only=False)[source]#\n\nReturn index of first occurrence of maximum over requested axis.\n\nNA/null values are excluded.\n\nParameters:\n\naxis{0 or ‘index’, 1 or ‘columns’}, default 0\n\nThe axis to use. 0 or ‘index’ for row-wise, 1 or ‘columns’ for column-wise.\n\nskipnabool, default True\n\nExclude NA/null values. If an entire row/column is NA, the result\nwill be NA.\n\nnumeric_onlybool, default False\n\nInclude onlyfloat,intorbooleandata.\n\nNew in version 1.5.0.\n\nReturns:\n\nSeries\n\nIndexes of maxima along the specified axis.\n\nRaises:\n\nValueError\n\nIf the row/column is empty\n\nNotes\n\nThis method is the DataFrame version ofndarray.argmax.\nndarray.argmax\nExamples\n\nConsider a dataset containing food consumption in

## Embedding generation

In [5]:
#### https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/

In [6]:
# need to update this
# !pip install --upgrade pyOpenSSL

In [7]:

def memory_check():
    # Get the memory allocated on the GPU
    allocated_memory = torch.cuda.memory_allocated()
    # Get the total memory on the GPU
    total_memory = torch.cuda.get_device_properties(0).total_memory
    # Get the free memory
    free_memory = total_memory - allocated_memory

    print(f"Allocated Memory: {allocated_memory / 1e6:.2f} MB")
    print(f"Free Memory: {free_memory / 1e6:.2f} MB")
    print(f"Total Memory: {total_memory / 1e6:.2f} MB")
    
memory_check()

Allocated Memory: 0.00 MB
Free Memory: 34079.90 MB
Total Memory: 34079.90 MB


## embedding models

In [8]:
# model_name = "thenlper/gte-large"
# embeddings = HuggingFaceEmbeddings(model_name=model_name)  

In [9]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"trust_remote_code": True}
)

In [10]:
memory_check()

Allocated Memory: 1745.80 MB
Free Memory: 32334.10 MB
Total Memory: 34079.90 MB


In [11]:
# #https://huggingface.co/dunzhang/stella_en_400M_v5


# model_name = "dunzhang/stella_en_400M_v5"
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs={"trust_remote_code": True}
# )

### Using ChromaDB

In [12]:
parent_doc_path = "/home/agrawal.bi/DS5500/vector_db/parent_docs"
child_doc_path = "../vector_db/child_docs"

In [13]:
# Paths to empty
parent_docs_path = parent_doc_path
child_docs_path = os.path.abspath(child_doc_path)

# Function to empty a folder
def empty_folder(folder):
    for item in os.listdir(folder):
        item_path = os.path.join(folder, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        except Exception as e:
            print(f'Failed to delete {item_path}. Reason: {e}')

# Empty the folders
empty_folder(parent_docs_path)
empty_folder(child_docs_path)

print("Folders emptied successfully.")

Folders emptied successfully.


In [14]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# The vectorstore to use to index the child chunks

In [15]:
fs = LocalFileStore(parent_doc_path)
store = create_kv_docstore(fs)

vectorstore = Chroma(collection_name= "split_parents", 
                     embedding_function= embeddings, 
                     persist_directory= child_doc_path)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

### adding pandas docs

In [16]:
# Adding docs to local storage
for doc in tqdm(pandas_docs, desc="Adding documents", unit="doc"):
    retriever.add_documents([doc], ids=None)

Adding documents: 100%|██████████| 2286/2286 [07:40<00:00,  4.96doc/s]


In [17]:
# Load Chroma vectorstore
loaded_vectorstore = Chroma(
    collection_name="split_parents",
    embedding_function=embeddings,
    persist_directory= child_doc_path
)
 
# Load parent document store
loaded_file_store = LocalFileStore(parent_doc_path)
 
# Recreate ParentDocumentRetriever
loaded_retriever = ParentDocumentRetriever(
    vectorstore=loaded_vectorstore,
    docstore=loaded_file_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [18]:
len(list(loaded_file_store.yield_keys()))

5135

In [24]:
loaded_vectorstore.similarity_search("how can I handle irregularly spaced time series data, including filling missing values?")

[Document(metadata={'doc_id': '2e013833-a3d8-424d-a721-3ad2da85805c', 'source': 'https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.fillna.html', 'title': 'pandas.core.resample.Resampler.fillna — pandas 2.2.2 documentation'}, page_content='Method to use for filling holes in resampled data\n\n‘pad’ or ‘ffill’: use previous valid observation to fill gap\n(forward fill).\n\n‘backfill’ or ‘bfill’: use next valid observation to fill gap.\n\n‘nearest’: use nearest valid observation to fill gap.\n\nlimitint, optional\n\nLimit of how many consecutive missing values to fill.\n\nReturns:\n\nSeries or DataFrame\n\nAn upsampled Series or DataFrame with missing values filled.\n\nReferences\n\nhttps://en.wikipedia.org/wiki/Imputation_(statistics)\n\nExamples'),
 Document(metadata={'doc_id': '34348068-f5e9-4a70-b2ef-a8ced5726c33', 'source': 'https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.interpolate.html', 'title': 'pandas.core.resample.Resampler.

In [25]:
loaded_retriever.invoke("how can I handle irregularly spaced time series data, including filling missing values?")

[b'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"metadata": {"source": "https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.fillna.html", "title": "pandas.core.resample.Resampler.fillna \\u2014 pandas 2.2.2 documentation"}, "page_content": "pandas.core.resample.Resampler.fillna#\\n\\nfinalResampler.fillna(method,limit=None)[source]#\\n\\nFill missing values introduced by upsampling.\\n\\nIn statistics, imputation is the process of replacing missing data with\\nsubstituted values[1]. When resampling data, missing values may\\nappear (e.g., when the resampling frequency is higher than the original\\nfrequency).\\n\\nMissing values that existed in the original data will\\nnot be modified.\\n\\nParameters:\\n\\nmethod{\\u2018pad\\u2019, \\u2018backfill\\u2019, \\u2018ffill\\u2019, \\u2018bfill\\u2019, \\u2018nearest\\u2019}\\n\\nMethod to use for filling holes in resampled data\\n\\n\\u2018pad\\u2019 or \\u2018ff

### adding scikit-learn docs

In [28]:
# Define the path
path = '../documents/processed_docs/scikit_learn_docs.pkl'

# Load data from a file
with open(path, 'rb') as file:
    scikit_learn_docs = pickle.load(file)

In [30]:
len(scikit_learn_docs)

937

In [41]:
# Adding docs to local storage
for doc in tqdm(scikit_learn_docs, desc="Adding documents", unit="doc"):
    retriever.add_documents([doc], ids=None)

In [40]:
# # Adding docs to local storage
# for doc in tqdm(scikit_learn_docs[918:], desc="Adding documents", unit="doc"):
#     retriever.add_documents([doc], ids=None)

Adding documents: 100%|██████████| 19/19 [00:32<00:00,  1.73s/doc]


In [45]:
# # Load Chroma vectorstore
# loaded_vectorstore = Chroma(
#     collection_name="split_parents",
#     embedding_function=embeddings,
#     persist_directory= child_doc_path
# )
 
# # Load parent document store
# loaded_file_store = LocalFileStore(parent_doc_path)
 
# # Recreate ParentDocumentRetriever
# loaded_retriever = ParentDocumentRetriever(
#     vectorstore=loaded_vectorstore,
#     docstore=loaded_file_store,
#     child_splitter=child_splitter,
#     parent_splitter=parent_splitter
# )

In [44]:
len(list(loaded_file_store.yield_keys()))

9488

In [51]:
loaded_vectorstore.similarity_search("hyperparameters for linear regression model")

[Document(metadata={'doc_id': '22bc2787-24eb-4bf1-b4e3-761414464810', 'source': 'https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_compare_gpr_krr.html', 'title': 'Comparison of kernel ridge and Gaussian process regression — scikit-learn 1.5.1 documentation'}, page_content='"kernel using default hyperparameters"\n)'),
 Document(metadata={'doc_id': '739d2906-aee5-4d02-bdc3-db84142a5433', 'source': 'https://scikit-learn.org/stable/modules/grid_search.html', 'title': '3.2. Tuning the hyper-parameters of an estimator — scikit-learn 1.5.1 documentation'}, page_content='3.2.Tuning the hyper-parameters of an estimator#\n\nHyper-parameters are parameters that are not directly learnt within estimators.\nIn scikit-learn they are passed as arguments to the constructor of the\nestimator classes. Typical examples includeC,kernelandgammafor Support Vector Classifier,alphafor Lasso, etc.\nCkernelgammaalpha\nIt is possible and recommended to search the hyper-parameter space for the\n

In [48]:
loaded_retriever.invoke("I am getting an error while try to normalize the data")

[b'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"metadata": {"source": "https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html", "title": "pandas.json_normalize \\u2014 pandas 2.2.2 documentation"}, "page_content": ">>> data = [\\n...     {\\n...         \\"id\\": 1,\\n...         \\"name\\": \\"Cole Volk\\",\\n...         \\"fitness\\": {\\"height\\": 130, \\"weight\\": 60},\\n...     },\\n...     {\\"name\\": \\"Mark Reg\\", \\"fitness\\": {\\"height\\": 130, \\"weight\\": 60}},\\n...     {\\n...         \\"id\\": 2,\\n...         \\"name\\": \\"Faye Raker\\",\\n...         \\"fitness\\": {\\"height\\": 130, \\"weight\\": 60},\\n...     },\\n... ]\\n>>> pd.json_normalize(data, max_level=0)\\n    id        name                        fitness\\n0  1.0   Cole Volk  {\'height\': 130, \'weight\': 60}\\n1  NaN    Mark Reg  {\'height\': 130, \'weight\': 60}\\n2  2.0  Faye Raker  {\'height\': 130, \'weight\': 60}\\n\\n\\n

### adding numpy docs

In [53]:
# Define the path
path = '../documents/processed_docs/numpy_docs.pkl'

# Load data from a file
with open(path, 'rb') as file:
    numpy_docs = pickle.load(file)

In [54]:
len(numpy_docs)

2464

In [55]:
# Adding docs to local storage
for doc in tqdm(numpy_docs, desc="Adding documents", unit="doc"):
    retriever.add_documents([doc], ids=None)

Adding documents: 100%|██████████| 2464/2464 [07:04<00:00,  5.80doc/s] 


In [56]:
len(list(loaded_file_store.yield_keys()))

13720

In [59]:
loaded_vectorstore.similarity_search("how can i create a 2d array?")

[Document(metadata={'doc_id': '631be762-2f59-49e2-a8f1-03975d3aa39b', 'source': 'https://numpy.org/doc/stable/reference/arrays.ndarray.html', 'title': 'The N-dimensional array (ndarray) — NumPy v2.0 Manual'}, page_content='>>> y = x[:,1]\n>>> y\narray([2, 5], dtype=int32)\n>>> y[0] = 9 # this also changes the corresponding element in x\n>>> y\narray([9, 5], dtype=int32)\n>>> x\narray([[1, 9, 3],\n       [4, 5, 6]], dtype=int32)\n\n\nConstructing arrays#\n\nNew arrays can be constructed using the routines detailed inArray creation routines, and also by using the low-levelndarrayconstructor:\n\nndarray(shape[,\xa0dtype,\xa0buffer,\xa0offset,\xa0...])\n\nAn array object represents a multidimensional, homogeneous array of fixed-size items.'),
 Document(metadata={'doc_id': '9c040906-0485-4022-83fd-0be4b34b81ff', 'source': 'https://numpy.org/doc/stable/user/absolute_beginners.html', 'title': 'NumPy: the absolute basics for beginners — NumPy v2.0 Manual'}, page_content='How to create a basic 

In [60]:
loaded_retriever.invoke("how can i create a 2d array?")

[b'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"metadata": {"source": "https://numpy.org/doc/stable/reference/arrays.ndarray.html", "title": "The N-dimensional array (ndarray) \\u2014 NumPy v2.0 Manual"}, "page_content": "The N-dimensional array (ndarray)#\\nndarray\\nAnndarrayis a (usually fixed-size) multidimensional\\ncontainer of items of the same type and size. The number of dimensions\\nand items in an array is defined by itsshape,\\nwhich is atupleofNnon-negative integers that specify the\\nsizes of each dimension. The type of items in the array is specified by\\na separatedata-type object (dtype), one of which\\nis associated with each ndarray.\\nshapetuple\\nAs with other container objects in Python, the contents of anndarraycan be accessed and modified byindexing or\\nslicingthe array (using, for example,Nintegers),\\nand via the methods and attributes of thendarray.\\n\\nDifferentndarrayscan share the same data, so that\

In [61]:
loaded_retriever

ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x2b245f985970>, docstore=<langchain.storage.file_system.LocalFileStore object at 0x2b2452958610>, child_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x2b2453657e20>, parent_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x2b2453657760>)